In [4]:
from sqlalchemy import MetaData
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
import sys
import json
from DB_setup import Protein_sequence, DBD, Experiment, PFM, TF_info, Base, DBD_TF_Maps
from sqlalchemy.orm import sessionmaker


#setting up database directory, database will be named tf.db for now
db_directory = 'C:\\Users\\Alex\\Documents\\fordyce_rotation\\tf.db'

#setting up engine to connect to the database, using sqlite
engine = create_engine('sqlite:///'+db_directory)
conn = engine.connect()
meta = MetaData()
meta.reflect(engine)

query = '''select * 
from protein_sequences 
inner join protein_information
on pid = tf_id
where protein_sequences.sequence isnull'''

pid_to_uniprot = {}
results = conn.execute(query)
for result in results: 
    pid_to_uniprot[result['pid']] = result['uniprot']

In [5]:
print(pid_to_uniprot)

{1153: '', 1155: '', 1159: '', 1160: '', 1161: '', 1162: '', 1163: '', 1186: '', 1202: '', 1223: '', 1291: '', 1293: '', 1294: '', 1328: '', 1329: '', 1330: '', 1339: '', 1341: '', 1348: '', 1356: '', 1357: '', 1358: '', 1359: '', 1362: '', 1377: '', 1379: '', 1380: '', 1381: '', 1386: ''}


In [ ]:
for prot, uni in pid_to_uniprot.items():
    if 'F1NSH4' in uni:
        uni = 'A0A1D5NT38'
    elif 'G66GR3' in uni:
        uni = 'Q66GR3'
    good_uni = uni.strip().upper()
    q = '''UPDATE protein_information
SET uniprot = \'%s\'
WHERE tf_id = %s;''' % (good_uni, prot)
    pid_to_uniprot[prot] = good_uni
    
    conn.execute(q)

In [ ]:
from uniprot_domains_sequences import get_sequences
searchable = []
for prot, uni in pid_to_uniprot.items():
    if uni:
        searchable.append(uni)
uniprot_seq = get_sequences(searchable, len(searchable))[0]

In [ ]:
pid_to_seq = {}
for prot,uni in pid_to_uniprot.items():
    if uni in uniprot_seq:
        pid_to_seq[prot] = uniprot_seq[uni]

In [ ]:
print(pid_to_seq)

In [ ]:
query2 = '''UPDATE protein_sequences
SET sequence = %s
WHERE pid = %s;'''

for pid, seq in pid_to_seq.items(): 
    query2 = '''UPDATE protein_sequences
SET sequence = \'%s\'
WHERE pid = %s;''' %(seq, pid)
    conn.execute(query2)
    